## Analysing workflow 1 for implementation
It is a simple flow for storing the input data (also known as primary series):  

CSV file as input -> dummy test (eg. check for missing data) -> transcode -> write in sqlite

- For this domain the input data are time series.
- There are several dataflows (collections) for the domain
    - the name of the dataflow is contained in the first column (dataflow)
    - for our example it is: EBSSBS_PRL_A
    - for now this will be the name of the sqlite file for this dataflow
    - for table name: originals
    - for columns keep original names, first column removed




In [5]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# load csv
df1 = pd.read_csv('./EBSSBS_PRL_A - SDMX-CSV template - V01.csv', sep=';')

# print first 5 lines (head)
print(df1.head())

# some statistics
df1.describe()

                  DATAFLOW FREQ  TIME_PERIOD  REF_AREA INDICATOR ACTIVITY  \
0  ESTAT:EBSSBS_PRL_A(1.0)    A          NaN       NaN       ENT        B   
1  ESTAT:EBSSBS_PRL_A(1.0)    A          NaN       NaN       ENT      B05   
2  ESTAT:EBSSBS_PRL_A(1.0)    A          NaN       NaN       ENT     B051   
3  ESTAT:EBSSBS_PRL_A(1.0)    A          NaN       NaN       ENT     B052   
4  ESTAT:EBSSBS_PRL_A(1.0)    A          NaN       NaN       ENT      B06   

  NUMBER_EMPL PRODUCT TURNOVER CLIENT_RESIDENCE  ...  UNIT_MEASURE UNIT_MULT  \
0          _T      _Z       _Z               _Z  ...            PN         0   
1          _T      _Z       _Z               _Z  ...            PN         0   
2          _T      _Z       _Z               _Z  ...            PN         0   
3          _T      _Z       _Z               _Z  ...            PN         0   
4          _T      _Z       _Z               _Z  ...            PN         0   

   DECIMALS  OBS_STATUS OBS_STATUS_1  CONF_STATUS  CONF_

,TIME_PERIOD,REF_AREA,OBS_VALUE,UNIT_MULT,DECIMALS,OBS_STATUS_1,CONF_STATUS,CONF_STATUS_1,DOMINANCE,SHARE_SECOND,COMMENT_OBS
count,0.0,0.0,0.0,6532.000000,6532.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,0.945193,0.0,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,1.393731,0.0,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,3.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,3.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# extract column names
print(df1.columns.tolist())
# get implicit column types
print(df1.dtypes)
# check for duplicates
df1.columns.has_duplicates

['DATAFLOW', 'FREQ', 'TIME_PERIOD', 'REF_AREA', 'INDICATOR', 'ACTIVITY', 'NUMBER_EMPL', 'PRODUCT', 'TURNOVER', 'CLIENT_RESIDENCE', 'OBS_VALUE', 'UNIT_MEASURE', 'UNIT_MULT', 'DECIMALS', 'OBS_STATUS', 'OBS_STATUS_1', 'CONF_STATUS', 'CONF_STATUS_1', 'DOMINANCE', 'SHARE_SECOND', 'COMMENT_OBS']
DATAFLOW             object
FREQ                 object
TIME_PERIOD         float64
REF_AREA            float64
INDICATOR            object
ACTIVITY             object
NUMBER_EMPL          object
PRODUCT              object
TURNOVER             object
CLIENT_RESIDENCE     object
OBS_VALUE           float64
UNIT_MEASURE         object
UNIT_MULT             int64
DECIMALS              int64
OBS_STATUS           object
OBS_STATUS_1        float64
CONF_STATUS         float64
CONF_STATUS_1       float64
DOMINANCE           float64
SHARE_SECOND        float64
COMMENT_OBS         float64
dtype: object


False

For now we will consider the following types for each column (as saved in sqlite)
also don't forget that SQLite is "typeless" (for more see, https://www.sqlite.org/datatypes.html):

1. FREQ                 VARCHAR(1) = 'A'
1. TIME_PERIOD          INT (with range)
1. REF_AREA             VARCHAR(2)
1. INDICATOR            VARCHAR(4)
1. ACTIVITY             VARCHAR(4)
1. NUMBER_EMPL          VARCHAR(7)
1. PRODUCT              VARCHAR(1) = '_Z'
1. TURNOVER             VARCHAR(1) = '_Z'
1. CLIENT_RESIDENCE     VARCHAR(1) = '_Z'
1. OBS_VALUE            FLOAT ? (not specified)
1. UNIT_MEASURE         VARCHAR(3)
1. UNIT_MULT            VARCHAR(1) in ['0', '3'] (or INT)
1. DECIMALS             VARCHAR(1) in ['0', '1', '2', '3', '4', '5', '6'] (or INT with range)
1. OBS_STATUS           VARCHAR(1)
1. OBS_STATUS_1         VARCHAR(1)
1. CONF_STATUS          VARCHAR(1)
1. CONF_STATUS_1        VARCHAR(1)
1. DOMINANCE            ? (not specified)
1. SHARE_SECOND         ? (not specified)
1. COMMENT_OBS          TEXT

The working types for each column as stored in pandas dataframes:

For now we will consider the following types for each column (as saved in sqlite)
also don't forget that SQLite is "typeless" (for more see, https://www.sqlite.org/datatypes.html):

1. FREQ                 VARCHAR(1) = 'A'
1. TIME_PERIOD          INT (with range)
1. REF_AREA             VARCHAR(2)
1. INDICATOR            VARCHAR(4)
1. ACTIVITY             VARCHAR(4)
1. NUMBER_EMPL          VARCHAR(7)
1. PRODUCT              VARCHAR(1) = '_Z'
1. TURNOVER             VARCHAR(1) = '_Z'
1. CLIENT_RESIDENCE     VARCHAR(1) = '_Z'
1. OBS_VALUE            FLOAT ? (not specified)
1. UNIT_MEASURE         VARCHAR(3)
1. UNIT_MULT            VARCHAR(1) in ['0', '3'] (or INT)
1. DECIMALS             VARCHAR(1) in ['0', '1', '2', '3', '4', '5', '6'] (or INT with range)
1. OBS_STATUS           VARCHAR(1)
1. OBS_STATUS_1         VARCHAR(1)
1. CONF_STATUS          VARCHAR(1)
1. CONF_STATUS_1        VARCHAR(1)
1. DOMINANCE            ? (not specified)
1. SHARE_SECOND         ? (not specified)
1. COMMENT_OBS          TEXT

In [54]:
# using numpy types: https://numpy.org/doc/stable/user/basics.types.html

# in can be a strict representation for some number of bytes
print(df1['FREQ'].astype(np.string_).head())
# here pandas decide the length of string
print(df1['ACTIVITY'].astype(np.string_).head())
# check for None or numpy.NaN in a column
print(sum(df1['FREQ'].astype(np.string_).isna()))
# check to see if all values in a column are a specified value
len(df1['FREQ']) == sum(df1['FREQ'] == 'A')


0    b'A'
1    b'A'
2    b'A'
3    b'A'
4    b'A'
Name: FREQ, dtype: bytes8
0       b'B'
1     b'B05'
2    b'B051'
3    b'B052'
4     b'B06'
Name: ACTIVITY, dtype: bytes72
0


True

In [55]:
# check possible categories for a category column
print(df1['ACTIVITY'].astype('category').cat.categories)


Index(['B', 'B05', 'B051', 'B052', 'B06', 'B061', 'B062', 'B07', 'B071',
       'B072',
       ...
       'R92', 'R920', 'R93', 'R931', 'R932', 'S95', 'S951', 'S952', 'S96',
       'S960'],
      dtype='object', length=343)


In [56]:

# change all column types based on a dictionary with column name type
df1.astype({
    'FREQ': 'S',     
    'TIME_PERIOD': pd.Int16Dtype(), # np.uint16, cannot use this type if column is nullable
    'REF_AREA': str,
    'INDICATOR': 'category'        
# ACTIVITY        
# NUMBER_EMPL     
# PRODUCT         
# TURNOVER        
# CLIENT_RESIDENCE
# OBS_VALUE       
# UNIT_MEASURE   
# UNIT_MEASURE 
# UNIT_MULT    
# DECIMALS     
# OBS_STATUS   
# OBS_STATUS_1 
# CONF_STATUS  
# CONF_STATUS_1
# DOMINANCE    
# SHARE_SECOND 
# COMMENT_OBS   
}, ).head()

,DATAFLOW,FREQ,TIME_PERIOD,REF_AREA,INDICATOR,ACTIVITY,NUMBER_EMPL,PRODUCT,TURNOVER,CLIENT_RESIDENCE,...,UNIT_MEASURE,UNIT_MULT,DECIMALS,OBS_STATUS,OBS_STATUS_1,CONF_STATUS,CONF_STATUS_1,DOMINANCE,SHARE_SECOND,COMMENT_OBS
0,ESTAT:EBSSBS_PRL_A(1.0),b'A',<NA>,nan,ENT,B,_T,_Z,_Z,_Z,...,PN,0,0,P,NaN,NaN,NaN,NaN,NaN,NaN
1,ESTAT:EBSSBS_PRL_A(1.0),b'A',<NA>,nan,ENT,B05,_T,_Z,_Z,_Z,...,PN,0,0,P,NaN,NaN,NaN,NaN,NaN,NaN
2,ESTAT:EBSSBS_PRL_A(1.0),b'A',<NA>,nan,ENT,B051,_T,_Z,_Z,_Z,...,PN,0,0,P,NaN,NaN,NaN,NaN,NaN,NaN
3,ESTAT:EBSSBS_PRL_A(1.0),b'A',<NA>,nan,ENT,B052,_T,_Z,_Z,_Z,...,PN,0,0,P,NaN,NaN,NaN,NaN,NaN,NaN
4,ESTAT:EBSSBS_PRL_A(1.0),b'A',<NA>,nan,ENT,B06,_T,_Z,_Z,_Z,...,PN,0,0,P,NaN,NaN,NaN,NaN,NaN,NaN


## Tasks

1. Extract a flow or (any kind of flows available?)
- input: input file from the folder in-data
- output: binary pandas.dataframe
- run: take the file and convert to pandas (one internal check: to have the mandatory columns?)

2. Transform with check (is doing a type conversion for each column, parametriseable?) and check restrictions for each column
- input: binary pandas
- output: binary pandas
- run: conversions and type domains checks

3. Another check (dummy, or better several dummies) to show the power of paralelism (if a check do not depand on the result of the previous check)
- input: binary pandas
- output: a sqlite staged record
- run: raise an error if not valid input or just not create output? 

4. No idea (I will create a dummy one to make a conversion x -> x: identity)
- input: binary pandas
- output: binary pandas
- run: conversions

5. Write SQLITE
- input: binary pandas
- output: write in output (not staged)
- run: transaction

6. Wrapper to archive the input

7. Wripper to archive the session

Questions:
1. give a try with staged and not staged
2. what happens if the input contains several flows (first column)
3. what happens if some input rows can pe imported and some not

Ideas:
1. because the entire domain  (all files: code and configurations) are versioned in log for each process
to add a checksum of the file used or direct the commit id

# SQLite
Trying some different approaches for an optimal storage as sql table.



In [4]:
# load a test file
# load csv
df1 = pd.read_csv('test_10_lines.csv', sep=';')

# print first 5 lines (head)
print(df1.head())

# some statistics
df1.describe()

                  DATAFLOW FREQ  TIME_PERIOD REF_AREA INDICATOR ACTIVITY  \
0  ESTAT:EBSSBS_PRL_A(1.0)    A         2020       LU      TOVT     C256   
1  ESTAT:EBSSBS_PRL_A(1.0)    A         2020       LU      EMPL     N783   
2  ESTAT:EBSSBS_PRL_A(1.0)    A         2021       LU      TOVT     M741   
3  ESTAT:EBSSBS_PRL_A(1.0)    A         2021       LU      TOVT     B091   
4  ESTAT:EBSSBS_PRL_A(1.0)    A         2020       LU      TOVT     M750   

  NUMBER_EMPL PRODUCT TURNOVER CLIENT_RESIDENCE  ...  UNIT_MEASURE UNIT_MULT  \
0      E10T19      _Z       _Z               _Z  ...           TRY         3   
1          _T      _Z       _Z               _Z  ...            PN         3   
2          _T      _Z       _Z               _Z  ...           CHF         3   
3     E50T249      _Z       _Z               _Z  ...           ALL         0   
4      EGE250      _Z       _Z               _Z  ...           EUR         3   

   DECIMALS  OBS_STATUS OBS_STATUS_1 CONF_STATUS CONF_STATUS_1

,TIME_PERIOD,OBS_VALUE,UNIT_MULT,DECIMALS,DOMINANCE,SHARE_SECOND,COMMENT_OBS
count,10.000000,10.000000,10.000000,10.000000,0.0,0.0,0.0
mean,2020.500000,431395.430889,1.500000,3.700000,NaN,NaN,NaN
std,0.849837,305797.755873,1.581139,2.002776,NaN,NaN,NaN
min,2019.000000,66214.112489,0.000000,0.000000,NaN,NaN,NaN
25%,2020.000000,140251.302975,0.000000,3.000000,NaN,NaN,NaN
50%,2020.500000,440998.254190,1.500000,4.000000,NaN,NaN,NaN
75%,2021.000000,687019.531108,3.000000,5.000000,NaN,NaN,NaN
max,2022.000000,880508.098648,3.000000,6.000000,NaN,NaN,NaN


In [6]:
# just create the table always
with sqlite3.connect("database.db") as conn:
    df1.to_sql("EBSSBS_PRL_A", conn, if_exists="replace")

In [7]:
# a little bit of analyse
with sqlite3.connect("database.db") as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name from sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("Tables:", tables)
    for table_name in tables:
        cursor.execute(f"PRAGMA table_info({table_name[0]});")
        columns = cursor.fetchall()
        print(f"Columns in table {table_name[0]}")
        for column in columns:
            print(column)
        cursor.execute(f"PRAGMA foreign_key_list({table_name[0]});")
        keys = cursor.fetchall()
        print(f"Keys in table {table_name[0]}")
        for key in keys:
            print(key)



Tables: [('EBSSBS_PRL_A',)]
Columns in table EBSSBS_PRL_A
(0, 'index', 'INTEGER', 0, None, 0)
(1, 'DATAFLOW', 'TEXT', 0, None, 0)
(2, 'FREQ', 'TEXT', 0, None, 0)
(3, 'TIME_PERIOD', 'INTEGER', 0, None, 0)
(4, 'REF_AREA', 'TEXT', 0, None, 0)
(5, 'INDICATOR', 'TEXT', 0, None, 0)
(6, 'ACTIVITY', 'TEXT', 0, None, 0)
(7, 'NUMBER_EMPL', 'TEXT', 0, None, 0)
(8, 'PRODUCT', 'TEXT', 0, None, 0)
(9, 'TURNOVER', 'TEXT', 0, None, 0)
(10, 'CLIENT_RESIDENCE', 'TEXT', 0, None, 0)
(11, 'OBS_VALUE', 'REAL', 0, None, 0)
(12, 'UNIT_MEASURE', 'TEXT', 0, None, 0)
(13, 'UNIT_MULT', 'INTEGER', 0, None, 0)
(14, 'DECIMALS', 'INTEGER', 0, None, 0)
(15, 'OBS_STATUS', 'TEXT', 0, None, 0)
(16, 'OBS_STATUS_1', 'TEXT', 0, None, 0)
(17, 'CONF_STATUS', 'TEXT', 0, None, 0)
(18, 'CONF_STATUS_1', 'TEXT', 0, None, 0)
(19, 'DOMINANCE', 'REAL', 0, None, 0)
(20, 'SHARE_SECOND', 'REAL', 0, None, 0)
(21, 'COMMENT_OBS', 'REAL', 0, None, 0)
Keys in table EBSSBS_PRL_A


In [9]:
# lets have the dataframe optimized from dtypes perspective
DTYPES = {
    'REF_AREA' : 'category',
    'INDICATOR' : 'category',
    'ACTIVITY' : 'category',
    'NUMBER_EMPL' : 'category',
    'UNIT_MEASURE' : 'category',
    'UNIT_MULT' : 'category',
    'DECIMALS' : 'category',
    'OBS_STATUS' : 'category',
    'CONF_STATUS' : 'category'
}
df2 = pd.read_csv('test_10_lines.csv', sep=';', dtype=DTYPES)
df2.info()
# just create the table always
with sqlite3.connect("database2.db") as conn:
    df2.to_sql("EBSSBS_PRL_A", conn, if_exists="replace")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   DATAFLOW          10 non-null     object  
 1   FREQ              10 non-null     object  
 2   TIME_PERIOD       10 non-null     int64   
 3   REF_AREA          10 non-null     category
 4   INDICATOR         10 non-null     category
 5   ACTIVITY          10 non-null     category
 6   NUMBER_EMPL       10 non-null     category
 7   PRODUCT           10 non-null     object  
 8   TURNOVER          10 non-null     object  
 9   CLIENT_RESIDENCE  10 non-null     object  
 10  OBS_VALUE         10 non-null     float64 
 11  UNIT_MEASURE      10 non-null     category
 12  UNIT_MULT         10 non-null     category
 13  DECIMALS          10 non-null     category
 14  OBS_STATUS        10 non-null     category
 15  OBS_STATUS_1      10 non-null     object  
 16  CONF_STATUS       10 non-null

In [10]:
# a little bit of analyse
with sqlite3.connect("database2.db") as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name from sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("Tables:", tables)
    for table_name in tables:
        cursor.execute(f"PRAGMA table_info({table_name[0]});")
        columns = cursor.fetchall()
        print(f"Columns in table {table_name[0]}")
        for column in columns:
            print(column)
        cursor.execute(f"PRAGMA foreign_key_list({table_name[0]});")
        keys = cursor.fetchall()
        print(f"Keys in table {table_name[0]}")
        for key in keys:
            print(key)


Tables: [('EBSSBS_PRL_A',)]
Columns in table EBSSBS_PRL_A
(0, 'index', 'INTEGER', 0, None, 0)
(1, 'DATAFLOW', 'TEXT', 0, None, 0)
(2, 'FREQ', 'TEXT', 0, None, 0)
(3, 'TIME_PERIOD', 'INTEGER', 0, None, 0)
(4, 'REF_AREA', 'TEXT', 0, None, 0)
(5, 'INDICATOR', 'TEXT', 0, None, 0)
(6, 'ACTIVITY', 'TEXT', 0, None, 0)
(7, 'NUMBER_EMPL', 'TEXT', 0, None, 0)
(8, 'PRODUCT', 'TEXT', 0, None, 0)
(9, 'TURNOVER', 'TEXT', 0, None, 0)
(10, 'CLIENT_RESIDENCE', 'TEXT', 0, None, 0)
(11, 'OBS_VALUE', 'REAL', 0, None, 0)
(12, 'UNIT_MEASURE', 'TEXT', 0, None, 0)
(13, 'UNIT_MULT', 'TEXT', 0, None, 0)
(14, 'DECIMALS', 'TEXT', 0, None, 0)
(15, 'OBS_STATUS', 'TEXT', 0, None, 0)
(16, 'OBS_STATUS_1', 'TEXT', 0, None, 0)
(17, 'CONF_STATUS', 'TEXT', 0, None, 0)
(18, 'CONF_STATUS_1', 'TEXT', 0, None, 0)
(19, 'DOMINANCE', 'REAL', 0, None, 0)
(20, 'SHARE_SECOND', 'REAL', 0, None, 0)
(21, 'COMMENT_OBS', 'REAL', 0, None, 0)
Keys in table EBSSBS_PRL_A


No important differences in sql representation between version with categories and one with strings. Always pandas when saving convert them to TEXT type in sqlite.


Now I want to have a constraint to make impossible to save twice a record. Simplest idea is to have a unique constraint on top of all fields:

In [ ]:
conn = sqlite3.connect("db_unique_1.db")
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE new_table (
    field1 TEXT,
    field2 INT,
    field3 REAL,
    timestamp DATETIME,
    UNIQUE (field1, field2, field3)
)
""")
